
https://towardsdatascience.com/a-gentle-introduction-on-market-basket-analysis-association-rules-fa4b986a40ce



In [0]:
from IPython.display import clear_output
# Import necessary library
import pandas as pd
import numpy as np
import nltk
import os
import nltk.corpus
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

path = "/content/drive/My Drive/CA683 Data Mining/Continuous Assignment/InsideAirbnb/data/"
calendar = pd.read_csv(path+"calendar.csv")
listings = pd.read_csv(path+"listings.csv")
reviews = pd.read_csv(path+"reviews.csv")
clear_output()

In [0]:
col_list = list(listings.columns)
# basicly the text information about the host
text_columns=['summary','space','decription',"neighborhood_overview","notes","transit","access","interaction","house_rules","host_about"]
# 
host_string_columns = ["host_name","host_since","amenities"]

host_location_columns = ["host_neighbourhood","street","neighbourhood","neighbourhood_cleansed","city","market","host_location",
                         "latitude","longitude"]

host_numeric_columns = ["host_id","host_acceptance_rate","host_response_time","host_response_rate","host_listings_count",
                        "host_total_listings_count","square_feet","price","weekly_price","monthly_price",
                        "security_deposit","cleaning_fee","guests_included","extra_people","number_of_reviews",
                        "number_of_reviews_ltm","calculated_host_listings_count","calculated_host_listings_count_entire_homes",
                        "calculated_host_listings_count_private_rooms","calculated_host_listings_count_shared_rooms",
                        "reviews_per_month"]

host_date_columns = ["first_review"]

host_nights_columns = col_list[67:75]

host_category_columns = ["host_is_superhost","host_has_profile_pic","host_identity_verified","property_type","room_type",
                         "instant_bookable","is_business_travel_ready","require_guest_profile_picture","require_guest_phone_verification"]

host_score_columns = ["review_scores_rating","review_scores_accuracy","review_scores_cleanliness","review_scores_checkin",
                      "review_scores_communication","review_scores_location","review_scores_value"
                      ]
host_other_columns = ["host_verifications",]

# Apriori Example
the Apriori Examples are not about our Airbnb Dataset, they are toturials from Andrew's colab or from other tutorials on internet.
Different cleaning process are required for different dataset.


In [0]:
import numpy as np 
import pandas as pd 
from mlxtend.frequent_patterns import apriori, association_rules 

In [0]:
data = pd.read_csv('/content/drive/My Drive/CA683 Data Mining/M3 Colab Files/Data/Online_Retail.csv',encoding = 'iso-8859-1')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01/12/2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01/12/2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01/12/2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01/12/2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01/12/2010 08:26,3.39,17850.0,United Kingdom


In [0]:
# Stripping extra spaces in the description 
data['Description'] = data['Description'].str.strip() 
  
# Dropping the rows without any invoice number 
data.dropna(axis = 0, subset =['InvoiceNo'], inplace = True) 
data['InvoiceNo'] = data['InvoiceNo'].astype('str') 
  
# Dropping all transactions which were done on credit 
data = data[~data['InvoiceNo'].str.contains('C')] 

# Transactions done in the United Kingdom 
basket_UK = (data[data['Country'] =="United Kingdom"] 
          .groupby(['InvoiceNo', 'Description'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('InvoiceNo')) 

In [0]:
data_UK = data[data['Country'] =="United Kingdom"]
data_g = data_UK.groupby(['InvoiceNo','Description'])['Quantity'].sum()

In [0]:
basket_UK.head(3)

Description,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,12 PENCILS TALL TUBE WOODLAND,12 PINK HEN+CHICKS IN BASKET,12 PINK ROSE PEG PLACE SETTINGS,12 RED ROSE PEG PLACE SETTINGS,15 PINK FLUFFY CHICKS IN BOX,15CM CHRISTMAS GLASS BALL 20 LIGHTS,16 PC CUTLERY SET PANTRY DESIGN,16 PIECE CUTLERY SET PANTRY DESIGN,18PC WOODEN CUTLERY SET DISPOSABLE,2 DAISIES HAIR COMB,2 PICTURE BOOK EGGS EASTER BUNNY,2 PICTURE BOOK EGGS EASTER CHICKS,2 PICTURE BOOK EGGS EASTER DUCKS,20 DOLLY PEGS RETROSPOT,200 BENDY SKULL STRAWS,200 RED + WHITE BENDY STRAWS,20713,20713 wrongly marked,3 BIRDS CANVAS SCREEN,3 BLACK CATS W HEARTS BLANK CARD,3 DRAWER ANTIQUE WHITE WOOD CABINET,3 GARDENIA MORRIS BOXED CANDLES,3 HEARTS HANGING DECORATION RUSTIC,3 HOOK HANGER MAGIC GARDEN,3 HOOK PHOTO SHELF ANTIQUE WHITE,...,sold as set on dotcom,sold as set on dotcom and amazon,sold as set/6 by dotcom,sold in set?,sold with wrong barcode,stock check,stock creditted wrongly,taig adjust,taig adjust no stock,temp adjustment,test,throw away,thrown away,thrown away-can't sell,thrown away-can't sell.,to push order througha s stock was,water damage,water damaged,website fixed,wet,wet boxes,wet damaged,wet pallet,wet rusty,wet/rusty,wet?,wrong barcode,wrong barcode (22467),wrong code,wrong code?,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
InvoiceNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
# Defining the hot encoding function to make the data suitable  
# for the concerned libraries 
def hot_encode(x): 
    if(x<= 0): 
        return 0
    if(x>= 1): 
        return 1
  
# Encoding the datasets 

In [0]:
basket_encoded = basket_UK.applymap(hot_encode) 
basket_UK = basket_encoded 

In [0]:
frq_items = apriori(basket_UK, min_support = 0.02, use_colnames = True) 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 

In [0]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
164,"(ROSES REGENCY TEACUP AND SAUCER, PINK REGENCY...",(GREEN REGENCY TEACUP AND SAUCER),0.029248,0.050032,0.026409,0.902930,18.047007,0.024945,9.786461
166,"(PINK REGENCY TEACUP AND SAUCER, GREEN REGENCY...",(ROSES REGENCY TEACUP AND SAUCER),0.030909,0.051264,0.026409,0.854419,16.666982,0.024824,6.516911
26,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.037658,0.050032,0.030909,0.820768,16.404818,0.029024,5.300218
171,"(JUMBO BAG PINK POLKADOT, JUMBO STORAGE BAG SUKI)",(JUMBO BAG RED RETROSPOT),0.027052,0.103814,0.021695,0.801980,7.725163,0.018887,4.525739
147,(PINK REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER),0.037658,0.051264,0.029248,0.776671,15.150368,0.027317,4.248161
...,...,...,...,...,...,...,...,...,...
179,(JUMBO BAG RED RETROSPOT),"(JUMBO SHOPPER VINTAGE RED PAISLEY, JUMBO STOR...",0.103814,0.027480,0.020570,0.198142,7.210375,0.017717,1.212834
136,(WHITE HANGING HEART T-LIGHT HOLDER),(NATURAL SLATE HEART CHALKBOARD),0.116027,0.065299,0.021963,0.189289,2.898808,0.014386,1.152940
160,(WHITE HANGING HEART T-LIGHT HOLDER),(WOODEN PICTURE FRAME WHITE FINISH),0.116027,0.057639,0.021641,0.186519,3.235999,0.014954,1.158430
144,(WHITE HANGING HEART T-LIGHT HOLDER),(PARTY BUNTING),0.116027,0.085387,0.020249,0.174515,2.043821,0.010341,1.107971


# Apriori Example2
this Example**from apyori import apriori** is much more easyier than Andrew's **from mlxtend.frequent_patterns import apriori**
but it contains NaN.

In [0]:
!pip install apyori
clear_output()

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from apyori import apriori

In [0]:
data = pd.read_csv("/content/drive/My Drive/CA683 Data Mining/Continuous Assignment/InsideAirbnb/data/order_data.csv",delimiter=" ",header=None)

In [0]:
data.head()

,0,1,2,3,4,5,6,7,8
0,toothpaste,brush,milk,cereals,honey,bread,butter,cheese,yogurt
1,milk,cereals,honey,bread,cheese,razor,gel,shampoo,NaN
2,milk,cereals,honey,cheese,soap,shampoo,NaN,NaN,NaN
3,honey,bread,butter,cheese,mouthwash,toothpaste,NaN,NaN,NaN
4,cereals,honey,bread,butter,gel,soap,NaN,NaN,NaN


In [0]:
data_list = []
for row in range(0,20):
  # data_list.append([str(data.values[row,column]) for column in range(0,9)])
  row = list(data.iloc[row].values.astype(str))
  data_list.append(trp(row,8))
  # data_list = trp(data_list,9)
algo = apriori(data_list,min_support = 0.25,min_confidence = 0.2,min_lift = 2,min_length=2)

In [0]:
list(algo)

[RelationRecord(items=frozenset({'brush', 'toothpaste'}), support=0.25, ordered_statistics=[OrderedStatistic(items_base=frozenset({'brush'}), items_add=frozenset({'toothpaste'}), confidence=1.0, lift=2.5), OrderedStatistic(items_base=frozenset({'toothpaste'}), items_add=frozenset({'brush'}), confidence=0.625, lift=2.5)]),
 RelationRecord(items=frozenset({'mouthwash', 'toothpaste'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset({'mouthwash'}), items_add=frozenset({'toothpaste'}), confidence=0.8571428571428572, lift=2.142857142857143), OrderedStatistic(items_base=frozenset({'toothpaste'}), items_add=frozenset({'mouthwash'}), confidence=0.7499999999999999, lift=2.142857142857143)]),
 RelationRecord(items=frozenset({'honey', 'butter', 'bread'}), support=0.25, ordered_statistics=[OrderedStatistic(items_base=frozenset({'butter'}), items_add=frozenset({'honey', 'bread'}), confidence=0.625, lift=2.0833333333333335), OrderedStatistic(items_base=frozenset({'honey', 'bre

In [0]:
data_list

# Apriori Example3

In [0]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori


dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

In [0]:
np.shape(dataset[0])

(6,)

In [0]:
np.shape(data_list[10])

(38,)

In [0]:
df

,Apple,Corn,Dill,Eggs,Ice cream,Kidney Beans,Milk,Nutmeg,Onion,Unicorn,Yogurt
0,False,False,False,True,False,True,True,True,True,False,True
1,False,False,True,True,False,True,False,True,True,False,True
2,True,False,False,True,False,True,True,False,False,False,False
3,False,True,False,False,False,True,True,False,False,True,True
4,False,True,False,True,True,True,False,False,True,False,False


In [0]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)

frequent_itemsets

,support,itemsets
0,0.8,(Eggs)
1,1.0,(Kidney Beans)
2,0.6,(Milk)
3,0.6,(Onion)
4,0.6,(Yogurt)
5,0.8,"(Eggs, Kidney Beans)"
6,0.6,"(Onion, Eggs)"
7,0.6,"(Milk, Kidney Beans)"
8,0.6,"(Onion, Kidney Beans)"
9,0.6,"(Yogurt, Kidney Beans)"


# Apply on Airbnb

## results from Example2, contains NaN, not suggest this way

In [0]:
# if an list length is not long enough, use this function. it can fill the list to special length with nan
def trp(l, n):
    # return l[:n] + ["'nan'"]*(n-len(l))
    return l[:n] + ['nan']*(n-len(l))

In [0]:
line = listings['amenities'][0][1:-1]
line = line.replace('"','')

In [0]:
amenities_list = line.split(',')

In [0]:
max_ = 0
for i in range(len(listings['amenities'])):
  line = listings['amenities'][0][1:-1]
  line = line.replace('"','')
  amenities_list = line.split(',')
  if(len(amenities_list)>max_):
    max_ = len(amenities_list)
max_

38

In [0]:
data_list = []
for i in range(len(listings['amenities'])):
  line = listings['amenities'][i][1:-1]
  line = line.replace('"','')
  amenities_list = line.split(',')
  # data_list.append(trp(row,max_))
  data_list.append(amenities_list)

In [0]:
algo = apriori(data_list,min_support = 0.25,min_confidence = 0.2,min_lift = 1,min_length=2)

In [0]:
association_results = list(algo)

In [0]:
type(association_results)

list

## Methods from Example1 & 3
doesn't contains Nan in results






In [0]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [0]:
listings['amenities'][0]

'{Internet,Wifi,Kitchen,"Free parking on premises","Smoking allowed",Breakfast,"Indoor fireplace",Heating,Washer,Dryer,"Smoke detector","Carbon monoxide detector","First aid kit","Fire extinguisher",Essentials,Shampoo,"Lock on bedroom door",Hangers,"Hair dryer",Iron,"Laptop friendly workspace","translation missing: en.hosting_amenity_49","translation missing: en.hosting_amenity_50","Hot water","Bed linens","Extra pillows and blankets","Ethernet connection",Microwave,"Coffee maker",Refrigerator,Dishwasher,"Dishes and silverware","Cooking basics",Oven,"Single level home","Patio or balcony","Garden or backyard","Host greets you"}'

1. transform the data format, 

from this

>'{Internet,Wifi,Kitchen,"Free parking on premises","Smoking allowed",Breakfast,"Indoor fireplace"...}'

to this

>['Internet',
 'Wifi',
 'Kitchen',
 'Free parking on premises',
 'Smoking allowed',
 'Breakfast',
 'Indoor fireplace',
 'Heating',
 'Washer',
 'Dryer',
 'Smoke detector'...]

In [0]:
data_list = []
for i in range(len(listings['amenities'])):
  line = listings['amenities'][i][1:-1]
  line = line.replace('"','')
  amenities_list = line.split(',')
  # data_list.append(trp(row,max_))
  data_list.append(amenities_list) # data_list is what we need to use for next step

2. Apply TransactionEncodeer, encode the data

In [0]:
te = TransactionEncoder()
te_ary = te.fit(data_list).transform(data_list)

In [0]:
df = pd.DataFrame(te_ary,columns=te.columns_)

In [0]:
df_de = df.drop(columns=['Essentials']) # 'de' means drop Essentials, because this item is not clear, I want see if remove it. the result looks like

3. Calculate the support of each transations

In [0]:
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
frequent_itemsets_de = apriori(df_de, min_support=0.6, use_colnames=True)

In [0]:
frequent_itemsets.head(10)

,support,itemsets
0,0.956032,(Essentials)
1,0.722739,(Hair dryer)
2,0.813190,(Hangers)
3,0.948376,(Heating)
4,0.770425,(Iron)
5,0.836596,(Kitchen)
6,0.601225,(Laptop friendly workspace)
7,0.732910,(Shampoo)
8,0.879908,(Smoke detector)
9,0.617193,(TV)


4. Calculate other metrics for ARM, and sort the value 

In [0]:
rules = association_rules(frequent_itemsets, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1532,"(Heating, Hangers, Shampoo, Wifi)",(Essentials),0.605162,0.956032,0.601116,0.993313,1.038995,0.022561,6.575008
764,"(Wifi, Hangers, Shampoo)",(Essentials),0.619600,0.956032,0.615225,0.992939,1.038605,0.022868,6.226977
654,"(Heating, Hangers, Shampoo)",(Essentials),0.624740,0.956032,0.619818,0.992122,1.037750,0.022547,5.581013
130,"(Hangers, Shampoo)",(Essentials),0.641146,0.956032,0.635787,0.991641,1.037247,0.022831,5.260016
196,"(Iron, Shampoo)",(Essentials),0.609865,0.956032,0.604287,0.990854,1.036423,0.021237,4.807175


In [0]:
rules = association_rules(frequent_itemsets, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules

In [0]:
rules_de = association_rules(frequent_itemsets_de, metric ="lift", min_threshold = 1) 
rules_de = rules_de.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules_de.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
588,"(Iron, Smoke detector, Hangers, Wifi)",(Heating),0.617303,0.948376,0.606256,0.982105,1.035565,0.020821,2.884821
461,"(Iron, Smoke detector, Kitchen)",(Heating),0.615115,0.948376,0.604069,0.982041,1.035498,0.020708,2.874599
362,"(Iron, Smoke detector, Hangers)",(Heating),0.637756,0.948376,0.626271,0.981993,1.035447,0.021439,2.866864
391,"(Smoke detector, Hangers, Kitchen)",(Heating),0.620912,0.948376,0.608772,0.980447,1.033817,0.019914,2.640275
376,"(Iron, Wifi, Hangers)",(Heating),0.662802,0.948376,0.649677,0.980198,1.033554,0.021092,2.607022
